In [5]:
from bs4 import BeautifulSoup
import urllib.request as REQ
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus']=False

In [6]:
# ! pip install folium

  Using cached folium-0.11.0-py2.py3-none-any.whl (93 kB)


In [7]:
# ! pip install folium
# 파이썬에서 지도를 사용할 수 있는(키 등록 필요 없는)
import folium

## *지도
- folium 라이브러리
- Marker() 사용시 한글 깨짐 문제
    - html로 저장

In [8]:
map = folium.Map(  location=(35.23272308926247, 129.08276451409182),
                   zoom_start=15               
                  )
mk = folium.Marker( location=(35.23272308926247, 129.08276451409182),
                  popup = 'PNU'
             )
mk1 = folium.Marker( location=(35.234677321753196, 129.07859099720366),
                  popup = 'PNU LIB'
             )
mk.add_to(map)
mk1.add_to(map)
map #save( 'a.html' ) # html로 저장하면 한글 안 깨진다

## [교통사고 데이터]

In [9]:
filepath = '../data/'
df = pd.read_csv( filepath +'도로교통공단_전국_사망교통사고정보(2018).csv', 
                  encoding='euc-kr', 
                  index_col=0 )
df.shape

(3657, 26)

In [10]:
# len( df.columns ) 26개 컬럼
df.columns

Index(['발생년월일시', '발생분', '주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수',
       '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '사고유형', '법규위반_대분류', '법규위반',
       '도로형태_대분류', '도로형태', '가해자_당사자종별_대분류', '가해자_당사자종별', '피해자_당사자종별_대분류',
       '피해자_당사자종별', '발생위치X_UTMK', '발생위치Y_UTMK', '경도', '위도'],
      dtype='object')

## 연습문제

In [ ]:
1. 부산에서 일어난 요일별
사망자수, 사상자수 를 출력하시요

2. 부산 사하구 지역의 교통사고 위치를
지도로 표시하고 각마커에는 법규위반내용
을 표시하시요

3. 부산에서 발생한 총 사망자수를 구하시요

4. 부산지역 월요일에 발생한 사상자수
사고유형 법규위반을 구하시요

5.횡단중 일어난 사고의 요일 
발생지시도, 법규위반, 피해자_당사자종별
을 구하시요

In [17]:
# 1. 부산에서 일어난 요일별
# 사망자수, 사상자수 를 출력하시요

df[df['발생지시도'] == '부산']['요일'].unique()

np.sort( df[df['발생지시도'] == '부산']['요일'].unique() )

array(['금', '목', '수', '월', '일', '토', '화'], dtype=object)

In [46]:
df[df['발생지시도'] == '부산'][['사망자수','사상자수','요일']].groupby('요일').sum().loc[['월','화','수','목','금','토','일']]

,사망자수,사상자수
요일,,
월,16,22
화,13,17
수,19,48
목,22,33
금,14,14
토,28,39
일,11,15


In [ ]:
# 2. 부산 사하구 지역의 교통사고 위치를
# 지도로 표시하고 각마커에는 법규위반내용
# 을 표시하시요

In [47]:
Saha = df[(df['발생지시도'] == '부산') & (df['발생지시군구'] == '사하구')][['위도','경도','법규위반']]
Saha

,위도,경도,법규위반
발생년,,,
2018,35.092209,128.960255,보행자 보호의무 위반
2018,35.106171,128.963631,안전거리 미확보
2018,35.064719,128.979537,신호위반


## (1) Series 인덱싱을 사용해서 map에 표시
####  DataFrame을 인덱싱 하면 Series가 된다
- sr.loc['']
- sr.iloc[ ]
- sr['부여된 인덱스']

In [84]:
for i,v in Saha.iterrows():
    print( v.loc['위도'], v.loc['경도'], v['법규위반'] )
#     print(v) # 컬럼    값

35.09220898 128.9602555 보행자 보호의무 위반
35.10617056 128.9636314 안전거리 미확보
35.06471892 128.9795375 신호위반


In [87]:
Saha = df[(df['발생지시도'] == '부산') & (df['발생지시군구'] == '사하구')]
map = folium.Ma( location=( Saha['위도'].mean(), Saha['경도'].mean() ),
                    zoom_start=13)

for i,v in Saha.iterrows():
    print( v.loc['위도'], v.loc['경도'], v['법규위반'] )
    mk = folium.Marker( location=(v.loc['위도'], v.loc['경도']) ,popup = f"{v['법규위반']}"  )         
    mk.add_to(map)
                       
map

35.09220898 128.9602555 보행자 보호의무 위반
35.10617056 128.9636314 안전거리 미확보
35.06471892 128.9795375 신호위반


## (2) DataFrame을 컬럼 인덱스 후 표시

In [110]:
# 2. 부산 사하구 지역의 교통사고 위치를 지도로 표시하고 각마커에는 법규위반내용을 표시하시요
Saha = df[(df['발생지시도'] == '부산') & (df['발생지시군구'] == '사하구')]

map = folium.Map(  location=( Saha['위도'].mean(), Saha['경도'].mean() ),
                   zoom_start=13               
                  )
for lat,long, vio in zip( Saha['위도'], Saha['경도'], Saha['법규위반'] ):
    mk = folium.Marker( location=(lat, long), popup = f'{vio}' )         
    mk.add_to(map)

# map.save('car_acc.html')
map

In [81]:
# 3. 부산에서 발생한 총 사망자수를 구하시요

df[df['발생지시도'] == '부산'][['사망자수']].sum()

사망자수    123
dtype: int64

In [92]:
df[df['발생지시도'] == '부산']['사망자수'].sum()

123

In [82]:
# 4. 부산지역 월요일에 발생한 사상자수,사고유형,법규위반을 구하시요
df[(df['발생지시도'] == '부산') & (df['요일'] == '월')][['사상자수','사고유형','법규위반']]

,사상자수,사고유형,법규위반
발생년,,,
2018,1,횡단중,안전운전 의무 불이행
2018,1,횡단중,안전운전 의무 불이행
2018,1,기타,안전운전 의무 불이행
2018,1,측면충돌,신호위반
2018,1,측면충돌,안전거리 미확보
2018,1,차도통행중,과속
2018,3,측면충돌,중앙선 침범
2018,1,횡단중,과속
2018,2,공작물충돌,안전운전 의무 불이행


In [75]:
# 5. 전체 지역에서 횡단중 일어난 사고의 요일 발생지시도, 법규위반, 피해자_당사자종별을 구하시요
crossdf = df[df['사고유형']=='횡단중']
crossdf[['요일','발생지시도','법규위반','피해자_당사자종별']]

,요일,발생지시도,법규위반,피해자_당사자종별
발생년,,,,
2018,수,서울,안전운전 의무 불이행,보행자
2018,일,서울,안전운전 의무 불이행,보행자
2018,일,서울,안전운전 의무 불이행,보행자
2018,월,서울,신호위반,보행자
2018,금,서울,과속,보행자
...,...,...,...,...
2018,목,경기,보행자 보호의무 위반,보행자
2018,금,경기,신호위반,보행자
2018,토,경기,안전운전 의무 불이행,보행자


In [146]:
# 발생지시도, 법규위반, 피해자_당사자종별을 구하시요
crossdf = df[df['사고유형']=='횡단중']
crossdf.pivot_table( index=['발생지시도','법규위반','피해자_당사자종별'])

경도      경상자수        발생년월일시        발생분  \
발생지시도 법규위반        피해자_당사자종별                                                  
강원    과속          보행자        128.595435  0.250000  2.018036e+09  26.500000   
      보행자 보호의무 위반 보행자        128.205232  0.000000  2.018056e+09  32.500000   
      신호위반        보행자        128.914197  0.000000  2.018042e+09  25.000000   
      안전운전 의무 불이행 보행자        128.383547  0.050000  2.018078e+09  27.050000   
경기    과속          보행자        127.009786  0.111111  2.018075e+09  30.388889   
...                                 ...       ...           ...        ...   
충북    과속          보행자        127.552675  0.400000  2.018058e+09   9.800000   
      보행자 보호의무 위반 보행자        128.369068  0.000000  2.018033e+09  50.000000   
      신호위반        보행자        128.053131  0.000000  2.018077e+09  38.000000   
      안전운전 의무 불이행 보행자        127.518027  0.043478  2.018068e+09  19.869565   
      중앙선 침범      보행자        127.480678  0.000000  2.018101e+09  10.000000   

                               발생위치X_UTMK    발생위치Y_UTMK  부상신고자수  사망자수  \
발생지시도 법규위반        피해자_당사자종별                                             
강원    과속          보행자        1.096895e+06  1.941204e+06     0.0   1.0   
      보행자 보호의무 위반 보행자        1.061900e+06  1.975802e+06     0.0   1.5   
      신호위반        보행자        1.124563e+06  1.974730e+06     0.0   1.0   
      안전운전 의무 불이행 보행자        1.077836e+06  1.975895e+06     0.0   1.0   
경기    과속          보행자        9.565819e+05  1.925680e+06     0.0   1.0   
...                                   ...           ...     ...   ...   
충북    과속          보행자        1.004680e+06  1.868743e+06     0.0   1.0   
      보행자 보호의무 위반 보행자        1.077344e+06  1.887557e+06     0.0   1.0   
      신호위반        보행자        1.049182e+06  1.893208e+06     0.0   1.0   
      안전운전 의무 불이행 보행자        1.001602e+06  1.850234e+06     0.0   1.0   
      중앙선 침범      보행자        9.982740e+05  1.855517e+06     0.0   1.0   

                                 사상자수         위도  중상자수  
발생지시도 법규위반        피해자_당사자종별                             
강원    과속          보행자        1.250000  37.464301   0.0  
      보행자 보호의무 위반 보행자        1.500000  37.779202   0.0  
      신호위반        보행자        1.000000  37.763755   0.0  
      안전운전 의무 불이행 보행자        1.150000  37.778375   0.1  
경기    과속          보행자        1.111111  37.328942   0.0  
...                               ...        ...   ...  
충북    과속          보행자        1.400000  36.816775   0.0  
      보행자 보호의무 위반 보행자        1.000000  36.983294   0.0  
      신호위반        보행자        1.000000  37.036032   0.0  
      안전운전 의무 불이행 보행자        1.043478  36.649952   0.0  
      중앙선 침범      보행자        1.000000  36.697644   0.0  

[75 rows x 11 columns]